## Import libraries

In [36]:
import nest_asyncio
import qdrant_client

from llama_index.core import Settings
from llama_index.core import PromptTemplate
from llama_index.llms.ollama import Ollama
from llama_index.core import StorageContext
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.embeddings.fastembed import FastEmbedEmbedding
from llama_index.core.postprocessor import SentenceTransformerRerank
from llama_index.vector_stores.qdrant import QdrantVectorStore

from IPython.display import Markdown, display
from dotenv import load_dotenv
import os

## Setup Asyncio

In [37]:
import nest_asyncio

nest_asyncio.apply()

## Define LLM, Embedding model and re-ranker model

In [ ]:

llm = Ollama(model="llama3.2:1b", request_timeout=120.0)

embed_model = FastEmbedEmbedding(model_name="BAAI/bge-large-en-v1.5")

rerank = SentenceTransformerRerank(model="BAAI/bge-reranker-base", top_n=2)

In [39]:
Settings.embed_model = embed_model
Settings.llm = llm

## Read the documents

In [40]:
input_dir_path = './docs/paul_graham'

loader = SimpleDirectoryReader(
            input_dir = input_dir_path,
            required_exts=[".txt"],
            recursive=True
        )
docs = loader.load_data()

## Set up the Qdrant vector database

In [41]:
client = qdrant_client.QdrantClient(host="localhost", port=6333)
vector_store = QdrantVectorStore(client=client,
                                 collection_name="document_chat")
storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(docs,
                                        storage_context=storage_context)

## Define the query engine and prompt template

In [42]:
query_engine = index.as_query_engine(similarity_top_k=4,
                                     node_postprocessors=[rerank])

template = """Context information is below.
              ---------------------
              {context_str}
              ---------------------
              Given the context information above I want you to think
              step by step to answer the query in a crisp manner. Incase 
              you don't know the answer say 'I don't know!'.
              
              Query: {query_str}
              
              Answer:"""

qa_prompt_tmpl = PromptTemplate(template)

query_engine.update_prompts(
    {"response_synthesizer:text_qa_template": qa_prompt_tmpl}
)

## Query the document

In [43]:
response = query_engine.query("""How did the structure of funding startups 
                                 in batches contribute to the success and 
                                 growth of the Y Combinator program and the
                                 startups involved?""")

In [ ]:
display(Markdown(str(response)))

## Generate the dataset for evaluation

### Load the knowledge base

In [45]:
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = DirectoryLoader("./docs/paul_graham/")

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1024, chunk_overlap=20)

documents = loader.load_and_split(text_splitter)

In [ ]:
documents[0].to_json()

## Setup the models

In [47]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

In [48]:
generator_llm = Ollama(model="phi3:3.8b")
critic_llm = Ollama(model="llama3.2:1b")

ollama_emb = OllamaEmbeddings(
    model="nomic-embed-text",
)

## Create Ragas Testset Generator

In [49]:
from ragas.testset.generator import TestsetGenerator
from ragas.testset.evolutions import simple, reasoning, multi_context

generator = TestsetGenerator.from_langchain(
    generator_llm=generator_llm,
    critic_llm=critic_llm,
    embeddings=ollama_emb
)

In [ ]:
## Import the csv file
import pandas as pd
test_df = pd.read_csv("./docs/test_data_paul_graham.csv")
test_df.head()

## Evaluate the RAG pipeline

In [51]:
def generate_response(query_engine, question):
    response = query_engine.query(question)
    return {
        "answer": response.response,
        "contexts": [c.node.get_content() for c in response.source_nodes],
    }

In [ ]:
distribution = {simple: 0.5, reasoning: 0.25, multi_context: 0.25}

total_distribution = sum(distribution.values())

if total_distribution != 1:
    raise ValueError("The sum of the distribution values must equal 1.")

try:
    testset = generator.generate_with_langchain_docs(documents,
                                                 test_size=min(10, len(documents)), # Adjust test_size to not exceed available documents
                                                 distributions=distribution,
                                                 raise_exceptions=False,
                                                 )
except Exception as e:
    print(f"An error occurred: {e}")
    print(f"Debugging logs: {e.debug_logs}")

test_df = testset.to_pandas().dropna()

In [ ]:
from datasets import Dataset
from tqdm.auto import tqdm

test_questions = test_df["question"].values

responses = [generate_response(query_engine, q) for q in tqdm(test_questions)]

dataset_dict = {
    "question": test_questions,
    "answer": [response["answer"] for response in responses],
    "contexts": [response["contexts"] for response in responses],
    "ground_truth": test_df["ground_truth"].values.tolist(),
}

ragas_eval_dataset = Dataset.from_dict(dataset_dict)

In [53]:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama

from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_correctness,
    context_recall,
    context_precision,
)

In [ ]:
metrics = [faithfulness, answer_correctness,
           context_recall, context_precision]

critic_llm = Ollama(model="llama3.2:1b")

ollama_emb = OllamaEmbeddings(model="nomic-embed-text")

evaluation_result = evaluate(
    llm=critic_llm,
    embeddings=ollama_emb,
    dataset=ragas_eval_dataset,
    metrics=metrics
)

In [ ]:
eval_scores_df = pd.DataFrame(evaluation_result.scores)
eval_scores_df.to_csv("./docs/evaluation_scores_paul_graham.csv", index=False)